# Synthetic Data Generation for Datasheet Extraction

An end-to-end pipeline designed to solve the scarcity of labeled training data in the electronics domain. The system procedurally generates high-fidelity synthetic datasheets (HTML/PDF) paired with structured JSON ground truth. It employs a 'Deep & Wide' strategy, covering both complex multi-condition tables (Active components) and standardized specifications (Passive components), to train state-of-the-art Document Understanding models for automated specification analysis.

In [7]:
import ollama
import json
import random
import os
import datetime
from jinja2 import Template

In [8]:
try:
    # --- שלב 2: שליחת בקשה (Prompt) למודל ---
    MODEL_TO_USE = 'llama3'  # ⚠️ חשוב: שנה לשם של מודל שבאמת מותקן אצלך

    print(f"--- 💬 שולח בקשה למודל: {MODEL_TO_USE} ---")
    
    response = ollama.chat(
        model=MODEL_TO_USE,
        messages=[
            {
                'role': 'user',
                'content': 'כתוב בדיחה קצרה על מתכנתים',
            },
        ]
    )

    print("--- 💡 תשובת המודל ---")
    print(response['message']['content'])

except Exception as e:
    print("\n--- ❌ אירעה שגיאה! ---")
    print(f"פרטי השגיאה: {e}")
    print("\n--- טיפים לפתרון ---")
    print("1. ודא שאפליקציית OLLAMA פועלת ברקע.")
    print(f"2. ודא שהמודל '{MODEL_TO_USE}' באמת מותקן (הר בטרמינל 'ollama list').")
    print("3. אם הוא לא מותקן, הרץ בטרמינל 'ollama pull <model_name>' (למשל 'ollama pull llama3').")

--- 💬 שולח בקשה למודל: llama3 ---

--- ❌ אירעה שגיאה! ---
פרטי השגיאה: model requires more system memory (4.6 GiB) than is available (3.7 GiB) (status code: 500)

--- טיפים לפתרון ---
1. ודא שאפליקציית OLLAMA פועלת ברקע.
2. ודא שהמודל 'llama3' באמת מותקן (הר בטרמינל 'ollama list').
3. אם הוא לא מותקן, הרץ בטרמינל 'ollama pull <model_name>' (למשל 'ollama pull llama3').


# Phase 1 Implementation: Synthetic Generation Pipeline Analysis

In [29]:
# ==========================================
# 1. CONFIGURATION
# ==========================================
OUTPUT_DIR = "output_samples_v2"

STYLES = [
    {"name": "Modern", "font": "Arial, sans-serif", "border": "1px solid #ddd", "header_bg": "#f8f9fa", "padding": "8px"},
    {"name": "Technical", "font": "'Times New Roman', serif", "border": "1px solid black", "header_bg": "#fff", "padding": "4px"},
    {"name": "Compact", "font": "Helvetica, sans-serif", "border": "1px solid #555", "header_bg": "#e0e0e0", "padding": "6px"}
]

# ==========================================
# TEMPLATE 1: COMPLEX (MOSFET/LDO)
# ==========================================
HTML_TEMPLATE_COMPLEX = """
<!DOCTYPE html>
<html>
<head>
    <style>
        body { font-family: {{ style.font }}; padding: 30px; background-color: white; }
        .header { border-bottom: 2px solid black; margin-bottom: 20px; }
        .header h1 { margin: 0; font-size: 24px; }
        table { border-collapse: collapse; width: 100%; margin-top: 20px; font-size: 12px; }
        th, td { border: {{ style.border }}; padding: {{ style.padding }}; vertical-align: top; }
        th { background-color: {{ style.header_bg }}; text-align: left; }
        .condition-cell { font-size: 0.85em; color: #555; width: 30%; }
        .value-cell { text-align: center; width: 8%; }
    </style>
</head>
<body>
    <div class="header">
        <h1>{{ meta.part_number }}</h1>
        <p>{{ meta.description }} | <strong>{{ meta.manufacturer }}</strong></p>
        <small>Date: {{ meta.date }} | Pkg: {{ meta.package }}</small>
    </div>
    <h3>Electrical Characteristics (Complex Active)</h3>
    <p><i>T<sub>A</sub> = 25°C unless otherwise noted.</i></p>
    <table>
        <thead>
            <tr>
                <th>Parameter</th>
                <th>Symbol</th>
                <th>Test Conditions</th>
                <th>Min</th>
                <th>Typ</th>
                <th>Max</th>
                <th>Unit</th>
            </tr>
        </thead>
        <tbody>
            {% for param in specs %}
            <tr>
                <td><b>{{ param.name }}</b></td>
                <td>{{ param.symbol }}</td>
                <td class="condition-cell">{{ param.condition | safe }}</td>
                <td class="value-cell">{{ param.min if param.min is not none else '-' }}</td>
                <td class="value-cell">{{ param.typ if param.typ is not none else '-' }}</td>
                <td class="value-cell">{{ param.max if param.max is not none else '-' }}</td>
                <td>{{ param.unit }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
</body>
</html>
"""

# ==========================================
# TEMPLATE 2: SIMPLE (Resistor/Capacitor)
# ==========================================
HTML_TEMPLATE_SIMPLE = """
<!DOCTYPE html>
<html>
<head>
    <style>
        body { font-family: {{ style.font }}; padding: 30px; background-color: white; }
        .header { border-bottom: 1px solid #ccc; margin-bottom: 20px; }
        .header h2 { margin: 0; color: #333; }
        table { border-collapse: collapse; width: 100%; margin-top: 20px; font-size: 14px; }
        th, td { border-bottom: 1px solid #ddd; padding: 12px; text-align: left; }
        th { background-color: #f9f9f9; }
    </style>
</head>
<body>
    <div class="header">
        <h2>{{ meta.part_number }} - {{ meta.description }}</h2>
        <small>{{ meta.manufacturer }} ({{ meta.date }})</small>
    </div>
    <h3>Specifications (Simple Passive)</h3>
    <table>
        <thead>
            <tr>
                <th>Parameter</th>
                <th>Symbol</th>
                <th>Value / Range</th>
                <th>Unit</th>
            </tr>
        </thead>
        <tbody>
            {% for param in specs %}
            <tr>
                <td>{{ param.name }}</td>
                <td>{{ param.symbol }}</td>
                <td>
                    {% if param.min is not none and param.max is not none %}
                        {{ param.min }} ... {{ param.max }}
                    {% elif param.typ is not none %}
                        {{ param.typ }}
                    {% elif param.max is not none %}
                        {{ param.max }} (max)
                    {% else %}
                        -
                    {% endif %}
                </td>
                <td>{{ param.unit }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
</body>
</html>
"""

# ==========================================
# GENERATOR CLASS
# ==========================================
class DatasheetGeneratorV2:
    def __init__(self):
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

    def render(self, data, style):
        if data.get("is_complex", True):
            template = Template(HTML_TEMPLATE_COMPLEX)
        else:
            template = Template(HTML_TEMPLATE_SIMPLE)
        return template.render(meta=data["metadata"], specs=data["specs"], style=style)

    def save(self, name, html, data):
        with open(f"{OUTPUT_DIR}/{name}.html", "w", encoding="utf-8") as f:
            f.write(html)
        
        clean_data = {
            "metadata": data["metadata"],
            "electrical_characteristics": data["specs"]
        }
        with open(f"{OUTPUT_DIR}/{name}.json", "w", encoding="utf-8") as f:
            json.dump(clean_data, f, indent=4)

    # ---------------------------------------------------------
    # FAMILY A: COMPLEX ACTIVE COMPONENTS
    # ---------------------------------------------------------
    def generate_complex_data(self, component_type=None):
        if component_type is None:
            component_type = random.choice(["MOSFET", "LDO"])

        specs = []
        
        if component_type == "MOSFET":
            vds_base = random.choice([30, 55, 60, 100, 600])
            specs.append({"name": "Drain-Source Breakdown Voltage", "symbol": "V(BR)DSS", "condition": "Vgs=0V, Id=250µA", "min": vds_base, "typ": None, "max": None, "unit": "V"})
            specs.append({"name": "Static Drain-Source On-Resistance", "symbol": "Rds(on)", "condition": "Vgs=10V, Id=20A<br>Vgs=4.5V, Id=15A", "min": None, "typ": 0.022, "max": 0.028, "unit": "Ω"})
            specs.append({"name": "Gate Threshold Voltage", "symbol": "Vgs(th)", "condition": "Vds=Vgs, Id=250µA", "min": 2.0, "typ": 3.0, "max": 4.0, "unit": "V"})
            specs.append({"name": "Zero Gate Voltage Drain Current", "symbol": "Idss", "condition": f"Vds={vds_base}V, Vgs=0V", "min": None, "typ": None, "max": 1.0, "unit": "µA"})
            specs.append({"name": "Gate-Body Leakage Current", "symbol": "Igss", "condition": "Vgs=±20V, Vds=0V", "min": None, "typ": None, "max": 100, "unit": "nA"})
            specs.append({"name": "Total Gate Charge", "symbol": "Qg", "condition": "Id=20A, Vds=0.5V(BR)DSS", "min": None, "typ": 45, "max": 60, "unit": "nC"})

            metadata = {"part_number": f"MOS-{random.randint(1000, 9999)}", "description": f"{vds_base}V N-Channel Power MOSFET", "package": "TO-220"}

        elif component_type == "LDO":
            vout = 3.3
            specs.append({"name": "Output Voltage", "symbol": "Vout", "condition": "Iout=1mA", "min": 3.2, "typ": 3.3, "max": 3.4, "unit": "V"})
            specs.append({"name": "Dropout Voltage", "symbol": "Vdo", "condition": "Iout=200mA", "min": None, "typ": 150, "max": 250, "unit": "mV"})
            specs.append({"name": "Quiescent Current", "symbol": "Iq", "condition": "Iout=0mA", "min": None, "typ": 40, "max": 75, "unit": "µA"})
            specs.append({"name": "Line Regulation", "symbol": "ΔVout/ΔVin", "condition": "Vin=4.3V to 6V", "min": None, "typ": 0.01, "max": 0.05, "unit": "%/V"})
            specs.append({"name": "Load Regulation", "symbol": "ΔVout/ΔIout", "condition": "Iout=1mA to 200mA", "min": None, "typ": 5, "max": 20, "unit": "mV"})

            metadata = {"part_number": f"LDO-{random.randint(1000, 9999)}", "description": "Low Dropout Regulator", "package": "SOT-23"}

        metadata["manufacturer"] = "Synthetic-Semi"
        metadata["date"] = str(datetime.date.today())
        
        return {"metadata": metadata, "specs": specs, "is_complex": True}

    # ---------------------------------------------------------
    # FAMILY B: SIMPLE PASSIVE COMPONENTS
    # ---------------------------------------------------------
    def generate_simple_data(self, component_type=None):
        if component_type is None:
            component_type = random.choice(["RESISTOR", "CAPACITOR"])

        specs = []
        
        if component_type == "RESISTOR":
            base_val = random.choice([10, 47, 100, 1000])
            specs.append({"name": "Resistance", "symbol": "R", "condition": "-", "min": round(base_val*0.99, 2), "typ": base_val, "max": round(base_val*1.01, 2), "unit": "Ω"})
            specs.append({"name": "Rated Power", "symbol": "Pd", "condition": "-", "min": None, "typ": 0.125, "max": None, "unit": "W"})
            specs.append({"name": "Max Working Voltage", "symbol": "Vmax", "condition": "-", "min": None, "typ": None, "max": 50, "unit": "V"})
            specs.append({"name": "Temperature Coefficient", "symbol": "TCR", "condition": "-", "min": -100, "typ": None, "max": 100, "unit": "ppm/°C"})
            
            desc = "Thick Film Chip Resistor"
            prefix = "RES"

        elif component_type == "CAPACITOR":
            base_val = random.choice([10, 22, 47, 100])
            unit = random.choice(["pF", "nF", "µF"])
            specs.append({"name": "Capacitance", "symbol": "C", "condition": "1kHz", "min": round(base_val*0.9, 1), "typ": base_val, "max": round(base_val*1.1, 1), "unit": unit})
            specs.append({"name": "Rated Voltage", "symbol": "Ur", "condition": "-", "min": None, "typ": None, "max": 16, "unit": "V"})
            specs.append({"name": "Dissipation Factor", "symbol": "DF", "condition": "1kHz", "min": None, "typ": None, "max": 5.0, "unit": "%"})
            specs.append({"name": "Insulation Resistance", "symbol": "IR", "condition": "Rated Voltage", "min": 10, "typ": None, "max": None, "unit": "GΩ"})
            specs.append({"name": "Operating Temperature", "symbol": "Top", "condition": "-", "min": -55, "typ": None, "max": 125, "unit": "°C"})

            desc = "Multilayer Ceramic Capacitor"
            prefix = "CAP"

        metadata = {
            "part_number": f"{prefix}-{random.randint(1000, 9999)}",
            "manufacturer": "Passive-Semi",
            "description": desc,
            "date": str(datetime.date.today())
        }

        return {"metadata": metadata, "specs": specs, "is_complex": False}

    def generate_random_component(self):
        family = random.choice(["COMPLEX", "SIMPLE"])
        if family == "COMPLEX": return self.generate_complex_data()
        else: return self.generate_simple_data()

# ==========================================
# 4. EXECUTION
# ==========================================
if __name__ == "__main__":
    gen = DatasheetGeneratorV2()
    print("Generator V2 Loaded Successfully.")

Generator V2 Loaded Successfully.


# VISUAL PLAYGROUND: Single Sample Preview

In [28]:
# ==========================================
# 🧪 VISUAL PLAYGROUND: Dual Family Preview
# ==========================================

from IPython.display import display, HTML
import json
import random

engine = DatasheetGeneratorV2()

# ---------------------------------------------------------
# 1. Family A (Complex)
# ---------------------------------------------------------
data_complex = engine.generate_complex_data()
style_a = random.choice(STYLES)
html_complex = engine.render(data_complex, style_a)

print(f"\n🔵 FAMILY A (Complex Active): {data_complex['metadata']['description']}")
display(HTML(html_complex))

# ---------------------------------------------------------
# 2. Family B (Simple)
# ---------------------------------------------------------
data_simple = engine.generate_simple_data()
style_b = random.choice(STYLES)
html_simple = engine.render(data_simple, style_b)

print(f"\n🟠 FAMILY B (Simple Passive): {data_simple['metadata']['description']}")
display(HTML(html_simple))

# ---------------------------------------------------------
# 3. JSON Preview
# ---------------------------------------------------------
print("\n🔍 JSON Example (Family A):")
print(json.dumps({"electrical_characteristics": data_complex["specs"]}, indent=2))


🔵 FAMILY A (Complex Active): 30V N-Channel Power MOSFET


Parameter,Symbol,Test Conditions,Min,Typ,Max,Unit
Drain-Source Breakdown Voltage,V(BR)DSS,"Vgs=0V, Id=250µA",30,-,-,V
Static Drain-Source On-Resistance,Rds(on),"Vgs=10V, Id=20AVgs=4.5V, Id=15A",-,0.022,0.028,Ω
Gate Threshold Voltage,Vgs(th),"Vds=Vgs, Id=250µA",2.0,3.0,4.0,V
Zero Gate Voltage Drain Current,Idss,"Vds=30V, Vgs=0V",-,-,1.0,µA
Gate-Body Leakage Current,Igss,"Vgs=±20V, Vds=0V",-,-,100,nA
Total Gate Charge,Qg,"Id=20A, Vds=0.5V(BR)DSS",-,45,60,nC



🟠 FAMILY B (Simple Passive): Thick Film Chip Resistor


Parameter,Symbol,Value / Range,Unit
Resistance,R,46.53 ... 47.47,Ω
Rated Power,Pd,0.125,W
Max Working Voltage,Vmax,50 (max),V
Temperature Coefficient,TCR,-100 ... 100,ppm/°C



🔍 JSON Example (Family A):
{
  "electrical_characteristics": [
    {
      "name": "Drain-Source Breakdown Voltage",
      "symbol": "V(BR)DSS",
      "condition": "Vgs=0V, Id=250\u00b5A",
      "min": 30,
      "typ": null,
      "max": null,
      "unit": "V"
    },
    {
      "name": "Static Drain-Source On-Resistance",
      "symbol": "Rds(on)",
      "condition": "Vgs=10V, Id=20A<br>Vgs=4.5V, Id=15A",
      "min": null,
      "typ": 0.022,
      "max": 0.028,
      "unit": "\u03a9"
    },
    {
      "name": "Gate Threshold Voltage",
      "symbol": "Vgs(th)",
      "condition": "Vds=Vgs, Id=250\u00b5A",
      "min": 2.0,
      "typ": 3.0,
      "max": 4.0,
      "unit": "V"
    },
    {
      "name": "Zero Gate Voltage Drain Current",
      "symbol": "Idss",
      "condition": "Vds=30V, Vgs=0V",
      "min": null,
      "typ": null,
      "max": 1.0,
      "unit": "\u00b5A"
    },
    {
      "name": "Gate-Body Leakage Current",
      "symbol": "Igss",
      "condition": "Vgs=\